### KP-Miner

In [1]:
import pke
def kpminer_extract(text):
    extractor = pke.unsupervised.KPMiner()
    df_counts = pke.load_document_frequency_file(input_file='../data/spark_df.tsv.gz')
    extractor.load_document(input=text, language='en')
    extractor.candidate_selection(lasf=1)
    extractor.candidate_weighting(df=df_counts)
    keyphrases = extractor.get_n_best()
    returned_keyphrases = []
    for keyphrase in keyphrases:
        keyphrase = keyphrase[0]
        returned_keyphrases.append(keyphrase)
    return ' '.join(returned_keyphrases)

In [2]:
project = 'spark'
method = 'kpminer'
import pandas as pd
import os
from tqdm import tqdm

df = pd.read_csv('../data/raw/test_{}.csv'.format(project))
kpminer_folder = '../data/keywords/{}/kpminer-idf/run_1/'
flag_content_df = pd.read_csv(f'../data/ablation/test_{project}_flag_content.csv')

if not os.path.exists(kpminer_folder.format(project)):
    os.makedirs(kpminer_folder.format(project))

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    bug_id = row['bug_id']
    
    if flag_content_df[flag_content_df['bug_id'] == bug_id]['run_flag'].values[0] == 0:
        continue
    
    if os.path.exists(os.path.join(kpminer_folder, f'{bug_id}.txt')):
        continue
    
    short_desc = row['short_desc']
    description = row['description']
    short_desc_keywords = kpminer_extract(short_desc)
    description_keywords = kpminer_extract(description)
    
    with open(kpminer_folder.format(project) + '/{}.txt'.format(bug_id), 'w') as f:
        f.write('Summary: {}'.format(short_desc_keywords))
        f.write('\n')
        f.write('Description: {}'.format(description_keywords))
        f.write('\n')  

100%|██████████| 2841/2841 [25:07<00:00,  1.88it/s]
